In [1]:
import cvxpy as cp
import numpy as np
import random

In [2]:
def queueGenerator(numQueue):
    listDataEquip, listDataPcard, listDataPtime, listDataPriority = [], [], [], []
    listEquip = ['4082', 'V93K']
    listPcard = ['AV44', 'FF44']

    for _ in range(numQueue):
        equip = random.choices(listEquip, weights=[0.8, 0.2])
        pcard = random.choices(listPcard, weights=[0.8, 0.2])
        # 0~100까지
        ptime = np.random.beta(1, 5, 1).item()*100
        priority = random.choices(range(0, 7), weights=[0.8, 0.01, 0.01, 0.01, 0.02, 0.05, 0.1])
        listDataEquip.append(equip)
        listDataPcard.append(pcard)
        listDataPtime.append(ptime)
        listDataPriority.append(priority)
    return listDataEquip, listDataPcard, listDataPtime, listDataPriority

numQueue = int(np.random.beta(1, 5, 1).item()*50 + 50)
listDataEquip, matrixPcard, p, priority = queueGenerator(numQueue)

In [3]:
len(listDataEquip)

53

In [ ]:
J = len(listDataEquip) # number of jobs
M = 14 # number of machines
r = 0.16 # release time
BIG_M = 1000
matrixEquip = np.array([[listDataEquip[j] == listDataEquip[m] for j in range(J)] for m in range(M)])
s = cp.Variable(J, nonneg=True)
z = cp.Variable((J, M), boolean=True) # 할당 여부
o = cp.Variable((J, J), boolean=True) # 순서 결정


# ----- 제약 조건 정의 -----
constraints = []

# 각 Job은 정확히 하나의 Machine에 할당
for j in range(J):
    constraints.append(cp.sum(z[j, :]) == 1)

# 장비 호환성
for j in range(J):
    for m in range(M):
        if not matrixEquip[m][j]:
            constraints.append(z[j, m] == 0)  
        if j == m:
            constraints.append(z[j, m] == 1)
            constraints.append(s[j] == 0)
            for k in range(M, J):
                constraints.append(o[j, k] == 1)

# 순서제약: 같은 machine에 할당된 job들만
for i in range(J):
    for j in range(J):
        if i == j:
            continue
        for m in range(M):
            constraints.append(
                s[j] >= s[i] + p[i] + r * (matrixPcard[i] != matrixPcard[j])
                - BIG_M * (3 - z[i, m] - z[j, m] - o[i, j])
            )
            constraints.append(
                s[i] >= s[j] + p[j] + r * (matrixPcard[i] != matrixPcard[j])
                - BIG_M * (3 - z[i, m] - z[j, m] - o[j, i])
            )
            # 동일 machine에 2개의 job이 있을 경우 반드시 순서가 존재해야함함 
            constraints.append(z[i, m] + z[j, m] <= 1 + o[i, j] + o[j, i])
        constraints.append(o[i, j] + o[j, i] <= 1)
            
# 우선순위 반영 (높은 priority가 먼저)
for i in range(M, J):
    for j in range(M, J):
        if priority[i] > priority[j]:
            constraints.append(o[i, j] == 1)
        elif priority[i] < priority[j]:
            constraints.append(o[j, i] == 1)

# ----- 목적 함수: 평균 완료 시간 최소화 -----
obj = cp.Minimize(cp.mean(s))

# ----- 문제 정의 및 풀기 -----
prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.HIGHS)

# ----- 결과 출력 -----
print('상태:', prob.status)
print('평균 Completion Time:', prob.value)

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

ganttData = []
for j in range(J):
    idxMachine = int(np.argmax(z.value[j], axis=0))
    
    # 직전 할당된 job과 pcard 차이가 있는지 확인인
    if (j >= M): 
        arrayZ = z[:, idxMachine].value
        idxBeforeJob = np.where(arrayZ[:j])[0][-1]
        if (matrixPcard[idxBeforeJob] != matrixPcard[j]):
            ganttData.append({
            'Job': f'Setup {j}',
            'Priority': priority[j],
            'Start': float(s.value[j] - r),
            'End': float(s.value[j]),
            'Machine': f'Machine {idxMachine}'
        })
    ganttData.append({
        'Job': f'Job {j}',
        'Priority': priority[j],
        'Start': float(s.value[j]),
        'End': float(s.value[j] + p[j]),
        'Machine': f'Machine {idxMachine}'
    })
dfGantt = pd.DataFrame(ganttData)

machine_list = sorted(dfGantt['Machine'].unique())
machine_to_y = {machine: idx for idx, machine in enumerate(machine_list)}

# Plotly 그래프 객체 초기화
fig = go.Figure()

# 각 작업/Setup을 개별 막대로 그림
for _, row in dfGantt.iterrows():
    fig.add_trace(go.Bar(
        x=[row['End'] - row['Start']],
        y=[machine_to_y[row['Machine']]],
        base=[row['Start']],
        orientation='h',
        name=row['Job'],
        hovertext=f"{row['Job']}: {row['Start']} ~ {row['End']} | {row['Priority']}",
        hoverinfo='text',
        marker_color='lightcoral' if 'Setup' in row['Job'] else 'skyblue',
        showlegend=False
    ))

# y축 설정
fig.update_yaxes(
    tickvals=list(machine_to_y.values()),
    ticktext=list(machine_to_y.keys()),
    title='Machine',
    autorange='reversed'
)

# x축 수치형으로 설정
fig.update_xaxes(title='Time (unit)', type='linear')

# 기타 레이아웃
fig.update_layout(
    title='Gantt Chart (with numeric x-axis using Plotly)',
    barmode='stack',
    height=300 + 50 * len(machine_to_y)
)

fig.show()

TypeError: 'NoneType' object is not subscriptable

In [8]:
dfGantt.head(40)

,Job,Priority,Start,End,Machine
0,Job 0,[0],0.000000,4.280379,Machine 0
1,Job 1,[0],0.000000,17.196877,Machine 1
2,Job 2,[0],0.000000,55.999969,Machine 2
3,Job 3,[0],0.000000,1.299649,Machine 3
4,Job 4,[0],0.000000,30.528691,Machine 4
5,Job 5,[0],0.000000,28.769651,Machine 5
6,Job 6,[6],0.000000,5.811779,Machine 6
7,Job 7,[0],0.000000,0.814001,Machine 7
8,Job 8,[0],0.000000,40.254640,Machine 8
9,Job 9,[5],0.000000,22.419921,Machine 9
